#### Question 1 : Define function description

In [11]:
from openai import OpenAI
client = OpenAI()

In [ ]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

In [ ]:
get_weather('berlin')

In [ ]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Fetch the Weather of the City",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Which city's weather ?"
            }
        },
        "required": ['city'],
        "additionalProperties": False
    }
}

In [ ]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

In [9]:
import chat_assistant
tools = chat_assistant.Tools()

In [ ]:
tools.add_tool(get_weather,get_weather_tool)

In [ ]:
tools.get_tools()

In [ ]:
developer_prompt = """
You are a weather assistant.

When provided with a city name, use the available tools to:
1. Fetch the current weather of the city.
2. Optionally add the weather information to the database.

After each response, follow up with a relevant question to keep the conversation going.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [ ]:
chat.run()

#### Question 2 : Adding another tool

In [ ]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [ ]:
set_weather_tool = {
    "type": "function",
    "name": "set_weather",
    "description": "Set the Weather of the City",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {"type": "string", "description": "Which city's weather?"},
            "temp": {"type": "number", "description": "Temperature in Fahrenheit?"}
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [ ]:
tools.add_tool(set_weather,set_weather_tool)

In [ ]:
tools.get_tools()

In [ ]:
known_weather_data

In [ ]:
chat.run()

#### Question 3 :

In [ ]:
!pip install fastmcp

In [ ]:
import importlib.metadata

version = importlib.metadata.version("fastmcp")
print("FastMCP version:", version)

#### MCP Server

In [2]:
!cd weather_server

/bin/bash: line 1: cd: weather_server: No such file or directory


In [5]:
import mcp_client

our_mcp_client = mcp_client.MCPClient(["python", "weather_mcp/weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

Started server with command: python weather_mcp/weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Weather MCP Server', 'version': '1.12.2'}}
Sending initialized notification...
Handshake completed successfully


In [6]:
our_mcp_client.get_tools()
our_mcp_client.call_tool('get_weather', {'city': 'Berlin'})

Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


{'content': [{'type': 'text', 'text': '20.0'}],
 'structuredContent': {'result': 20.0},
 'isError': False}

In [7]:
import json

class MCPTools:
    def __init__(self, mcp_client):
        self.mcp_client = mcp_client
        self.tools = None
    
    def get_tools(self):
        if self.tools is None:
            mcp_tools = self.mcp_client.get_tools()
            self.tools = convert_tools_list(mcp_tools)
        return self.tools

    def function_call(self, tool_call_response):
        function_name = tool_call_response.name
        arguments = json.loads(tool_call_response.arguments)

        result = self.mcp_client.call_tool(function_name, arguments)

        return {
            "type": "function_call_output",
            "call_id": tool_call_response.call_id,
            "output": json.dumps(result, indent=2),
        }

In [ ]:
our_mcp_client = mcp_client.MCPClient(["python", "weather_mcp/weather_server.py"])

our_mcp_client.start_server()
our_mcp_client.initialize()
our_mcp_client.initialized()

mcp_tools = mcp_client.MCPTools(mcp_client=our_mcp_client)


developer_prompt = """
You help users find out the weather in their cities. 
If they didn't specify a city, ask them. Make sure we always use a city.
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=mcp_tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

chat.run()

Started server with command: python weather_mcp/weather_server.py
Sending initialize request...
Initialize response: {'protocolVersion': '2024-11-05', 'capabilities': {'experimental': {}, 'prompts': {'listChanged': False}, 'resources': {'subscribe': False, 'listChanged': False}, 'tools': {'listChanged': True}}, 'serverInfo': {'name': 'Weather MCP Server', 'version': '1.12.2'}}
Sending initialized notification...
Handshake completed successfully


You: Berlin


Retrieving available tools...
Available tools: ['get_weather', 'set_weather']
Calling tool 'get_weather' with arguments: {'city': 'Berlin'}


You: Chennai ?


Calling tool 'get_weather' with arguments: {'city': 'Chennai'}


Calling tool 'get_weather' with arguments: {'city': 'Chennai'}


You: philadelphia


Calling tool 'get_weather' with arguments: {'city': 'Philadelphia'}
